<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>Proyecto de clasificación multiclase</font></h1>

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección vamos a desarrollar y evaluar modelos de redes neuronales para problemas de clasificación multiclase. Así, veremos
* Cómo preparar datos de clasificación multiclase para modelar con redes neuronales.
* Cómo evaluar modelos de redes neuronales de Keras con scikit-learn.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Importar clases, funciones y conjunto de datos</font>

Podemos comenzar importando todas las clases, funciones y dataset que necesitaremos en este tutorial. 

In [2]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold

# load dataset
dataframe=pd.read_csv("iris.csv",header=None)
#con .values se transforma el dataframe de pandas a un array de numpy
dataset=dataframe.values
X=dataset[:,0:4].astype(float)
y=dataset[:,4]

In [3]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [4]:
y

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versic

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Iris](https://archive.ics.uci.edu/ml/datasets/Iris)

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Codificar la variable de salida</font>

Como tenemos un problema de clasificación multiclase recordemos que debemos de utilizar One-Hot Encoding para poder formatear la salida. 

Por ejemplo, en este problema los tres valores de clase:
```
    Iris-setosa
    Iris-versicolor
    Iris-virginica
```

Por lo que convertimos esta salida en una codificación binaría como:
```
    Iris-setosa, Iris-versicolor, Iris-virginica
        1,             0,              0
        0,             1,              0
        0,             0,              1
```

Por tanto, 
1. Codificando primero las cadenas de manera coherente en números enteros utilizando la clase `LabelEncoder` de scikit-learn. 
2. Luego, conviertimos el vector de números enteros en One-Hot Enconding usando la función de Keras `to_categorical()`.

In [5]:
# encode class values as integers
#estas lineas de codigo pemiten transformar los tipos de flores (categorias) a numeros enteros
encoder=LabelEncoder()
encoder.fit(y)
encoded_y=encoder.transform(y)
print(encoded_y)
# convert integers to dummy variables (i.e. one hot encoded)
#transforma los numeros enteros del codigo anterior a vectores de unos y seros, como numeros binarios
dummy_y=to_categorical(encoded_y)
print(dummy_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Definir la red neuronal</font>

A continuación se muestra el procedimiento a la hora de crear la función a trabajar:
1. Crea una red simple completamente conectada con una capa oculta de 8 neuronas. 
2. La capa oculta utiliza una función de activación ReLu. 
3. Debido a que utilizamos One-Hot Encoding, la capa de salida debe crear 3 valores de salida, uno para cada clase. 
4. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología quedaría así:
```
    4 inputs -> [8 hidden nodes] -> 3 outputs
```
5. Tendremos una función de activación Softmax en la capa de salida. 
6. Finalmente, la red utiliza Adam con una función de pérdida logarítmica (`categorical_crossentropy`).

In [6]:
# define baseline model

#si tenemos un problema de multiclase, osea nuestra salida es de mas de dos opciones, se debe ocupar una funcion de activación
#softmax

#para la funcion de perdida en caso de una red que clasifique en dos salidas se usa una funcion de perdida binary_crossentropy
#y para una red que clasifique en tres o mas categoria se una la funcion de perdida categorical_crossentropy

def baseline_model():
    model=Sequential()
    model.add(Dense(8,input_dim=4,activation="relu"))
    model.add(Dense(3,activation="softmax"))
    # Compile model
    model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
    return model

Ahora podemos crear nuestro `KerasClassifier` para usarlo en scikit-learn. 

Aquí, pasamos el número de épocas como 200 y el tamaño de batch como 5 para usar al entrenar el modelo. 

In [7]:
estimator=KerasClassifier(build_fn=baseline_model,epochs=100,batch_size=5,verbose=0)

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Evaluar el modelo</font>

Ahora podemos evaluar nuestro modelo _(estimator)_ en nuestro conjunto de datos (`X` e `dummy_y`). Utilizando un procedimiento de validación cruzada de 10 veces (`kfold`)

In [8]:
kfold=KFold(n_splits=10,shuffle=True)
result=cross_val_score(estimator,X,dummy_y,cv=kfold)
print(f"Accuracy: {float(result.mean())*100:.2f} y desvacion {float(result.std())*100:.2f}")

C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(
C:\Users\56965\AppData\R

Accuracy: 96.00 y desvacion 4.42
